<a href="https://colab.research.google.com/github/LC1332/Luotuo-Chinese-LLM/blob/main/notebook/LAIONclean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 对于MMC4和LAION的清理工作

因为想训一个更好的CLIP，我们不得不在Image Coco Caption之外，去寻找更大的训练集。

这里就不可避免要去考虑MC4和LAION的清理工作

## Source

这里我们可以去考虑Hugging Face上面的LAION 2B

https://huggingface.co/datasets/laion/laion2b-multi-vit-l-14-embeddings/tree/main

而且居然有一个菩萨大神已经求了所有图片的embedding

另一方面是可以去考虑

https://github.com/allenai/mmc4



## 要收集哪些数据

另一方面我们要去考虑的是，要收集哪些数据。其实这里我们希望以CLIP-vit-l-14的词表为例子

如果词表上有一个input已经累积了很多数据，我们就不再考虑增加新的数据了

这样感觉更为合理一些，特别是对于我们初期这种小水管的训练来说

所以这个文档的第一步，我们先来分析VIT-l-14的词表

从这个链接可以看到 vit-l-14 这个模型的用法

https://huggingface.co/openai/clip-vit-large-patch14

让我们先跑上最基础的测试代码

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.8 MB/s eta 0:00:00


In [2]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

# url = "http://images.cocodataset.org/val2017/000000039769.jpg"
# image = Image.open(requests.get(url, stream=True).raw)

# inputs = processor(text=["a photo of a cat", "a photo of a dog"], images=image, return_tensors="pt", padding=True)

# outputs = model(**inputs)
# logits_per_image = outputs.logits_per_image # this is the image-text similarity score
# probs = logits_per_image.softmax(dim=1) # we can take the softmax to get the label probabilities


`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


当然我们后期可能是真的回去要到完整的CLIP模型的

但是我们在初步我们要的是CLIP的tokenizer

你会发现其实这个tokenizer是存储在processor里面的


In [3]:
print(processor)

CLIPProcessor:
- image_processor: CLIPImageProcessor {
  "crop_size": {
    "height": 224,
    "width": 224
  },
  "do_center_crop": true,
  "do_convert_rgb": true,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "feature_extractor_type": "CLIPFeatureExtractor",
  "image_mean": [
    0.48145466,
    0.4578275,
    0.40821073
  ],
  "image_processor_type": "CLIPImageProcessor",
  "image_std": [
    0.26862954,
    0.26130258,
    0.27577711
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

- tokenizer: CLIPTokenizerFast(name_or_path='openai/clip-vit-large-patch14', vocab_size=49408, model_max_length=77, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<|startoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedTok

In [4]:
from transformers import CLIPTokenizer

tokenizer = processor.tokenizer

input = tokenizer.tokenize("a dog is runing")

我们来测试一下这个tokenize的运行时间

已知 某个tokenizer可以使用 input = tokenizer.tokenize( string ) 来进行调用。设计10个英文句子，对这10个句子进行tokenize，并测量平均每次的运行时间

In [5]:
sentences = ["I love Python.", "Python is a great language.", "This is a cat.", 
             "There are many schools in this area.", "The sky is blue.",
             "Mary is a bright student.", "The book is on the table.", 
             "He likes to play football.", "She is a doctor.",
             "The food at this restaurant is delicious."]

import time

total_time = 0
num_sentences = len( sentences )

# 对10个句子进行tokenize，并测量平均每次的运行时间
for sentence in sentences:
  start_time = time.time()
  input = tokenizer.tokenize(sentence)
  end_time = time.time()
  total_time += (end_time - start_time)

avg_time = total_time / num_sentences
print("平均运行时间： %.5f 秒" % avg_time)

平均运行时间： 0.00033 秒


这里我在T4的机器下平均运行时间是  0.00016 秒 ， 让我们来关一下GPU，重新运行一次看看。 是0.00033秒。

看来在tokenizer这一步，是否使用GPU是不要紧的。

因为我们之前已经翻译了coco captioning，让我们来载入coco captioning，来统计一下不同token的出现频率。

